# Assignment 1 - Intro a Pandas

## Parte 1
El codigo que sigue carga el dataset de las olimpiadas (olympics.csv), que ha sido descargado de la entrada en Wikipedia en [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), y hace una limpieza de datos basica. 

Las columnas estan organizadas como # de Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Usen este dataset para responder las preguntas.

In [6]:
import pandas as pd

df = pd.read_csv('../data/olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Pregunta 0 (Ejemplo)

Cual es el primer pais en df?

*Esta funcion deberia regresar una Series.*

In [7]:
def answer_zero():
    # Esta funcion regresa la fila para Afghanistan, que es un Series object. 
    return df.iloc[0].name

# Pueden examinar que regresa su funcion llamandola en esta celda. 
answer_zero()

'Afghanistan'

### Pregunta 1
Que pais ha ganado la mayor cantidad de medallas de oro en los summer games?

*Esta funcion deberia retornar un solo valor de tipo string*

In [8]:
def answer_one():
    return print(df.iloc[df['Gold'].argmax()].name + ' es el pais que mas medallas ha ganado en los summer games')

answer_one()

United States es el pais que mas medallas ha ganado en los summer games


### Pregunta 2
Que pais tuvo la mayor diferencia en su conteo de medallas de oro entre summer y winter?


*Esta funcion deberia retornar un solo valor de tipo string*

In [10]:
def answer_two():
    return print( df.iloc[((df['Gold'] - df['Gold.1']).abs()).argmax()].name + ' es el pais que mayor diferencia en su conteo de medallas de oro entre summer y winter ha tenido')

answer_two()

United States es el pais que mayor diferencia en su conteo de medallas de oro entre summer y winter ha tenido


### Pregunta 3
Que pais tiene la mayor diferencia entre su conteo de summer gold medals y winter gold medals relativo a su total conteo de gold medals?

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Solo incluyan paises que tienen al menos 1 gold en summer **Y** winter.

*Esta funcion deberia retornar un solo valor de tipo string*

In [22]:
def answer_three():
    df_p3 = df.copy()
    df_p3 = df_p3[(df_p3['Gold']!=0) & (df_p3['Gold.1']!=0)]
    df_p3 = df_p3.drop(columns=['# Summer', '# Winter', '# Games', 'Silver', 'Silver.1', 'Silver.2', 'Bronze', 'Bronze.1', 'Bronze.2', 'Total', 'Total.1', 'Combined total', 'ID'])
    df_p3['var_aux_p3'] = ((df_p3['Gold'] - df_p3['Gold.1']) / df_p3['Gold.2']).abs()
    
    return print(df_p3.iloc[df_p3['var_aux_p3'].argmax()].name + ' es el pais que tiene la mayor diferencia entre su conteo de summer gold medals y winter gold medals relativo a su total conteo de gold medals (se realizo valor absoluto de la operacion para que no importe la direccion summer-winter o winter-summer)')

answer_three()

Bulgaria es el pais que tiene la mayor diferencia entre su conteo de summer gold medals y winter gold medals relativo a su total conteo de gold medals (se realizo valor absoluto de la operacion para que no importe la direccion summer-winter o winter-summer)


### Pregunta 4
Escriban una funcion que cree una Series llamada "Points" que sea un valor ponderado donde cada gold medal (`Gold.2`) cuenta por 3 puntos, silver medals (`Silver.2`) por 2 puntos, y bronze medals (`Bronze.2`) por 1 punto. La funcion deberia retornar unicamente la columna (un objeto Series) que han creado.

*Esta funcion deberia retornar una Series llamada `Points` de largo 146*

In [25]:
def answer_four():
    df_p4 = df.copy()
    df_p4['points'] = df_p4['Gold.2']*3 + df_p4['Silver.2']*2 + df_p4['Bronze.2']*1
    return df_p4.loc[:,'points']

answer_four()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: points, Length: 146, dtype: int64

## Part 2

Para el siguiente set de preguntas, vamos a usar data de censo de [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties son subdivisiones politicas y geograficas de estados en los Estados Unidos. Este dataset contiene data poblacional para counties y states en EEUU del 2010 al 2015. [Vea este documento](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) para una descripcion de los nombres de las variables.

El dataset census (census.csv) debe ser cargado como `census_df`. 

### Pregunta 5
Que estado tiene la mayor cantidad de counties? (tip: considere la llave sumlevel cuidadosamente! Tambien la necesitara para futuras preguntas...)

*Esta funcion deberia retornar un solo valor de tipo string*

In [27]:
import pandas as pd
census_df = pd.read_csv('../data/census.csv', index_col=0, skiprows=0)
census_df.head()

,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
SUMLEV,,,,,,,,,,,,,,,,,,,,,
40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,4801108,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [28]:
def answer_five():
    return print(census_df.iloc[census_df['COUNTY'].argmax(),4] + ' es el estado con más subdivisiones politicas y geograficas de los Estados Unidos ')

answer_five()

Virginia es el estado con más subdivisiones politicas y geograficas de los Estados Unidos 


### Pregunta 6

Solo viendo a los tres counties con mayor poblacion para cada estado, cuales son los tres estados con mayor poblacion (en orden de la poblacion mas alta a la mas baja)? Use `CENSUS2010POP`.

*Esta funcion debe retornar una lista de valores string.*

In [35]:
 def answer_six():
    df_p6 = census_df.copy()
    df_p6 = df_p6.groupby('STNAME', as_index=False).apply(lambda x: x.nlargest(3, 'CENSUS2010POP'))
    df_p6_sum_top3 = (df_p6.groupby(['STNAME'])['CENSUS2010POP'].agg('sum')).nlargest(3)
    df_p6_sum_top3 = df_p6_sum_top3.reset_index()
    
    return df_p6_sum_top3 #df_p6_sum_top3 #df_p6[df_p6.STNAME == df_p6_sum_top3.iloc[0,1]] #df_p6_sum_top3.iloc[0] #df_p6[df_p6.STNAME == 'California']

answer_six()

,STNAME,CENSUS2010POP
0,California,50167874
1,Texas,31606159
2,New York,24113524


### Pregunta 7
Que county ha tenido el cambio absoluto mas grande en poblacion dentro del periodo 2010-2015? (Tip: Valores de poblacion estan guardados entre las columnas POPESTIMATE2010 y POPESTIMATE2015, deben considerar las seis columnas.)

e.i. Si County Population en el periodo de 5 anios es 100, 120, 80, 105, 100, 130, entonces su cambio mas grande en el period seria |130-80| = 50

*Esta funcion deberia retornar un valor de tipo string.*

In [36]:
def answer_seven():
    df_p7 = census_df.copy()
    df_p7['minValue'] = df_p7.loc[:, ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']].min(axis=1)
    df_p7['maxValue'] = df_p7.loc[:, ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015']].max(axis=1)
    df_p7['deltaValue'] = (df_p7['maxValue'] - df_p7['minValue']).abs()
    
    return print(df_p7.iloc[df_p7['deltaValue'].argmax(),4] + ' es el país que ha tenido el cambio absoluto más grande en población dentro del período 2010-2015')

answer_seven()

Texas es el país que ha tenido el cambio absoluto más grande en población dentro del período 2010-2015


### Pregunta 8
En este datafile, los Estados Unidos esta partido en cuatro regiones usando la columna "REGION".

Creen un query que encuentre los counties que pertenezcan a la region 1 o 2, cuyo nombre empiece con "Washington", y cuyo POPESTIMATE2015 fue mayor que su POPESTIMATE 2014.

*Esta funcion debe retornar un DataFrame 5x2 con las columnas = ['STNAME', 'CTYNAME'] y el mismo indice ID que el census_df (en orden ascendient por indice).*

In [44]:
def answer_eight():
    df_p8 = census_df.copy()
    df_p8['var_aux'] = df_p8['POPESTIMATE2015'] > df_p8['POPESTIMATE2014']
    df_p8 = df_p8[df_p8['CTYNAME'].apply(lambda x: x.startswith('Washington'))]
    df_p8 = df_p8[((df_p8.REGION == 1) | (df_p8.REGION == 2)) & (df_p8.var_aux == True)]
    df_p8 = df_p8.filter(['STNAME', 'CTYNAME'])
    return df_p8

answer_eight()

,STNAME,CTYNAME
SUMLEV,,
50,Iowa,Washington County
50,Minnesota,Washington County
50,Pennsylvania,Washington County
50,Rhode Island,Washington County
50,Wisconsin,Washington County
